<a href="https://colab.research.google.com/github/kazaknii/MagicOfMemo_bot/blob/main/TheMagicOfMemoAPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 「メモの魔力」webアプリ構築
chatGPT APIを使って「メモの魔力」の内容を出力してくれるwebアプリを構築します。

## 1. 必要なライブラリのインストール
・　ChatGPT APIを使用するために必要なライブラリ「openai」

・　UI作成に必要なライブラリ「Streamlit」

・　確認用URL生成に必要なライブラリ「ngrok」

In [ ]:
!pip install streamlit==1.20.0 --quiet
!pip install pyngrok==4.1.1 --quiet
!pip install openai

streamlit、ngrok、openaiをインポートします。

In [35]:
import streamlit as st
from pyngrok import ngrok
import openai

## 2. コード作成
`%%writefile`のマジックコマンドを使って、チャットボットのコードを「app.py」に書き込みます。  
Streamlitでは、値を保持するために「st.session_state」を使います。  
https://docs.streamlit.io/library/api-reference/session-state

In [ ]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
import openai
import secret_keys  # 外部ファイルにAPI keyを保存

openai.api_key = secret_keys.openai_api_key

memo_prompt = """# [{Goal}: ユーザーが入力した{既存のアイデア}を横展開して{新たなアイデア}を出力する]
##変数 
- {既存のアイデア} 
- {新たなアイデア}
##メソッド
 - 既存のアイデア}のファクト：自分が興味を持ったことや発見した具体的な事実を５つ抽出して箇条書きにする 
STEP1：ファクトを抽象化：具体的な事実から導き出せる応用可能な法則や概念化して５つ箇条書きにする
STEP2：{新たなアイデア}:抽象化した法則をもとに、全く他のことに応用できるアイデアにして５つ箇条書きにする
STEP3：{新たなアイデア}をさらに掘り下げて、例として具体的なアイデアに落とし込んで５つ箇条書きにする
##実行"""

# st.session_stateを使いメッセージのやりとりを保存
if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {"role": "system", "content": memo_prompt}
        ]

# チャットボットとやりとりする関数
def communicate():
    messages = st.session_state["messages"]

    user_message = {"role": "user", "content": st.session_state["user_input"]}
    messages.append(user_message)

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )  

    bot_message = response["choices"][0]["message"]
    messages.append(bot_message)

    st.session_state["user_input"] = ""  # 入力欄を消去


# ユーザーインターフェイスの構築
st.title("メモの魔力アプリ")
st.write("アイデアのエッセンスを抽出して別のアイデアに発展させるbotです")
st.image("title.png")

user_input = st.text_input("既存のアイデアを入力してください 例）家事代行、ミニ豚カフェ、ケバブ屋台etc", key="user_input", on_change=communicate)

if st.session_state["messages"]:
    messages = st.session_state["messages"]

    for message in reversed(messages[1:]):  # 直近のメッセージを上に
        speaker = "🙂"
        if message["role"]=="assistant":
            speaker="🧠"

        st.write(speaker + ": " + message["content"])

## 3. 必要なKEYの設定

### OpenAIのAPI keyを設定
ChatGPT APIを使用するために必要な「API key」を設定します。  
`%%writefile`のマジックコマンドを使って、API keyを設定するコードを「secret_keys.py」に書き込みます。  
以下のコードの、  
`Your API key`の箇所を、自分のAPI keyにしてください。


In [36]:
%%writefile secret_keys.py

openai_api_key = "Your API key"

Overwriting secret_keys.py


*※API keyは他者に知られないように、慎重に扱ってください。*

### ngrokのAuthtokenを設定
ngrokで接続するために必要な「Authtoken」を設定します。  
以下のコードの、  
`!ngrok authtoken YourAuthtoken`  
における  
`YourAuthtoken`の箇所を、自分のAuthtokenに置き換えます。  
※Authtokenは、ngrokのサイトに登録すれば取得することができます。  
https://ngrok.com/


In [ ]:
!ngrok authtoken YourAuthtoken

*※ngrokのAuthtokenは他者に知られないように慎重に扱ってください。*

## 4. キービジュアルをアップロード
title.pngをアップロードします

## 5. アプリの起動
streamlitの`run`コマンドでアプリを起動します。


In [38]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

## 6. 確認用URLの生成（ngrok）

ngrokのプロセスを終了した上で、新たにポートを指定して接続します。  
接続の結果、urlを取得できます。  
※ngrokの無料プランでは同時に1つのプロセスしか動かせないので、
エラーが発生した場合は「ランタイム」→「セッションの管理」で不要な
Google Colabのセッションを修了しましょう。

In [ ]:
ngrok.kill()  # プロセスの修了
url = ngrok.connect(port="8501")  # 接続

URLのhttpの部分をhttpsに変換する関数を設定します。

In [40]:
def convert_http_to_https(url):
    if url.startswith("http://"):
        url = url.replace("http://", "https://", 1)
    return url

変換したurlを表示し、リンク先でチャットボットが動作することを確認します。

In [ ]:
print(convert_http_to_https(url))

生成されたURLにアクセスして動作を確認してください。